<a href="https://colab.research.google.com/github/arogya-gyawali/brainscan_AI/blob/main/notebooks/001_convert_nii_to_jpg_nii.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# Load libraries for reading .nii files, working with arrays, and saving images
!pip install nibabel
import nibabel as nib     # For loading .nii MRI files
import numpy as np        # For working with numerical arrays
import os                 # For file path handling
from PIL import Image     # For saving images
import matplotlib.pyplot as plt  # Optional: to view slices inline
import pandas as pd       # For handling metadata


In [22]:
# Input folder where the .nii files are stored
input_folder = "/content/drive/MyDrive/BrainScanAI/IXI-T1"

# Output folder where the .jpg will be stored
output_folder = "/content/drive/MyDrive/BrainScanAI/healthy_converted"
os.makedirs(output_folder, exist_ok=True)


print(f"✅ Output folder is ready: {output_folder}")


✅ Output folder is ready: /content/drive/MyDrive/BrainScanAI/healthy_converted


In [23]:
# List only .nii files, skip hidden files like .DS_Store
file_list = sorted([f for f in os.listdir(input_folder) if f.endswith(".nii") and not f.startswith('.')])
print(f"✅ Found {len(file_list)} .nii files")


✅ Found 581 .nii files


In [24]:
# Define how many slices to take before and after the center slice
# So total = 2 before + 1 center + 2 after = 5 slices per file
num_slices_each_side = 2

# Initialize a list to collect metadata for each saved image
metadata = []

# Loop through each .nii file in the folder
for i, file_name in enumerate(file_list):
    file_path = os.path.join(input_folder, file_name)

    try:
        # Load the .nii MRI volume using nibabel
        nii_image = nib.load(file_path)

        # Convert 3D volume data into a NumPy array
        image_data = nii_image.get_fdata()

        # Get the number of slices (along the z-axis / depth of the brain)
        z_dim = image_data.shape[2]

        # Find the center slice index
        mid_slice = z_dim // 2

        # Extract 5 slices: 2 before, the center, and 2 after
        for offset in range(-num_slices_each_side, num_slices_each_side + 1):
            slice_index = mid_slice + offset

            # Skip if the slice index is outside bounds
            if slice_index < 0 or slice_index >= z_dim:
                continue

            # Extract the 2D slice from the 3D volume
            slice_img = image_data[:, :, slice_index]

            # Normalize pixel values to range 0–255 for saving as image
            norm_img = 255 * (slice_img - np.min(slice_img)) / np.ptp(slice_img)
            norm_img = norm_img.astype(np.uint8)

            # Generate output file name with slice number
            base_name = os.path.splitext(file_name)[0]
            output_filename = f"{base_name}_slice{slice_index}.jpg"
            output_path = os.path.join(output_folder, output_filename)

            # Save the normalized slice as a grayscale .jpg image
            Image.fromarray(norm_img).save(output_path)

            # Record metadata for this image
            metadata.append({
                "file": output_filename,
                "label": 0,  # 0 = no tumor (healthy brain)
                "source": "IXI",  # Origin of this MRI
                "slice_index": slice_index  # For reference/debugging
            })

        # Print progress every 25 files
        if i % 25 == 0:
            print(f"✅ Processed {i}/{len(file_list)}: {file_name}")

    except Exception as e:
        # Catch and report any errors during processing
        print(f"❌ Error processing {file_name}: {e}")

# Save the metadata as a CSV file in the output folder
metadata_df = pd.DataFrame(metadata)
csv_path = os.path.join(output_folder, "no_tumor_metadata.csv")
metadata_df.to_csv(csv_path, index=False)

print("🏁 All .nii files processed and metadata saved.")


✅ Processed 0/581: IXI002-Guys-0828-T1.nii
✅ Processed 25/581: IXI038-Guys-0729-T1.nii
✅ Processed 50/581: IXI064-Guys-0743-T1.nii
✅ Processed 75/581: IXI090-Guys-0800-T1.nii
✅ Processed 100/581: IXI115-Guys-0738-T1.nii
✅ Processed 125/581: IXI143-Guys-0785-T1.nii
✅ Processed 150/581: IXI173-HH-1590-T1.nii
✅ Processed 175/581: IXI200-Guys-0812-T1.nii
✅ Processed 200/581: IXI228-Guys-0822-T1.nii
✅ Processed 225/581: IXI256-HH-1723-T1.nii
✅ Processed 250/581: IXI286-Guys-0859-T1.nii
✅ Processed 275/581: IXI312-Guys-0887-T1.nii
✅ Processed 300/581: IXI338-HH-1971-T1.nii
✅ Processed 325/581: IXI370-Guys-0921-T1.nii
✅ Processed 350/581: IXI396-HH-2115-T1.nii
✅ Processed 375/581: IXI422-Guys-1071-T1.nii
✅ Processed 400/581: IXI447-Guys-0979-T1.nii
✅ Processed 425/581: IXI475-IOP-1139-T1.nii
✅ Processed 450/581: IXI500-Guys-1017-T1.nii
✅ Processed 475/581: IXI531-Guys-1057-T1.nii
✅ Processed 500/581: IXI559-HH-2394-T1.nii
✅ Processed 525/581: IXI589-Guys-1080-T1.nii
✅ Processed 550/581: IXI61